In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_cliente"
TABELA_DESTINO = "v_credit.gold.dm_cliente"

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_cliente"),
    F.col("nm_cliente"),
    F.col("ds_email"),
    F.col("nm_regiao"),
    F.col("nu_idade").cast("smallint"),
    
    F.when(F.col("nu_idade") < 25, "Geração Z (Até 24)")
    .when((F.col("nu_idade") >= 25) & (F.col("nu_idade") <= 40), "Jovem Adulto (25-40)")
    .when((F.col("nu_idade") > 40) & (F.col("nu_idade") <= 60), "Adulto (41-60)")
    .otherwise("Senior (60+)").alias("ds_faixa_etaria")
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_cliente = s.cd_cliente"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Cliente carregada!")